In [1]:
import pennylane as qml

dev = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)

In [2]:
@qml.qnode(dev)
def quantum_function(x, theta):
    qml.Displacement(x, 0, wires=0)
    qml.Beamsplitter(theta, 0, wires=[0, 1])
    return qml.expval(qml.NumberOperator(0))

In [3]:
quantum_function(1., 0.543)

tensor(0.73301326, requires_grad=True)

In [8]:
import numpy as np

In [9]:
def generate_haar_unitary(dim):
    """Generate a (dim, dim) Haar-random matrix using the QR decomposition.
    Source: https://pennylane.ai/qml/demos/tutorial_haar_measure.html

    Parameters:
        dim (int): the dimension of the matrix.
    """
    # Step 1
    A, B = np.random.normal(size=(dim, dim)), np.random.normal(size=(dim, dim))
    Z = A + 1j * B

    # Step 2
    Q, R = np.linalg.qr(Z)

    # Step 3
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(dim)])

    # Step 4
    return np.dot(Q, Lambda)

In [71]:
n_modes = 5
cutoff_dim = 3

dev = qml.device("strawberryfields.fock", wires=n_modes, cutoff_dim=cutoff_dim)

# Define the random unitaries
self_unitary_top = generate_haar_unitary(3)
self_unitary_bottom = generate_haar_unitary(2)

@qml.qnode(dev)
def circuit():
    mid_l = 2
    mid_h = 3

    # Two-photon entanglement source
    qml.FockState(1, wires=mid_l)
    qml.FockState(1, wires=mid_h)

    # Beam splitters for entanglement
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi /4, wires=[mid_l - 1, mid_l])
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi /4, wires=[mid_h, mid_h + 1])

    # SWAP operation
    #qml.SWAP(wires=[mid_l, mid_h])

    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 2, wires=[mid_l, mid_h])
    # Interferometer (apply random Haar unitaries)
    
    qml.InterferometerUnitary(self_unitary_top, wires=list(range(mid_l + 1)))
    qml.InterferometerUnitary(self_unitary_bottom, wires=list(range(mid_h, n_modes)))

    # Measurement
    return qml.probs(wires=range(n_modes))

probs1 = circuit().reshape([cutoff_dim] * n_modes)
#print(probs)
# Fock states to measure at output




In [66]:
probs.shape

(3, 3, 3, 3, 3)

In [67]:
print(qml.draw(circuit)())

0: ────────────────────────────────────╭U(M0)─┤ ╭Probs
1: ──────╭BS(0.79,1.57)────────────────├U(M0)─┤ ├Probs
2: ──|1⟩─╰BS(0.79,1.57)─╭BS(1.57,1.57)─╰U(M0)─┤ ├Probs
3: ──|1⟩─╭BS(0.79,1.57)─╰BS(1.57,1.57)─╭U(M1)─┤ ├Probs
4: ──────╰BS(0.79,1.57)────────────────╰U(M1)─┤ ╰Probs


In [72]:
measure_states1 = [(2, 0, 0, 0, 0), (0, 2, 0, 0, 0), (0, 0, 2, 0, 0), (0, 0, 0, 2, 0), (0, 0, 0, 0, 2), (1, 1, 0, 0, 0), (1, 0, 1, 0, 0),
                  (1, 0, 0, 1, 0), (1, 0, 0, 0, 1), (0, 1, 1, 0, 0), (0, 1, 0, 1, 0), (0, 1, 0, 0, 1), (0, 0, 1, 1, 0), (0, 0, 1, 0, 1),
                  (0, 0, 0, 1, 1)]

# extract the probabilities of calculating several
# different Fock states at the output, and print them out
for i in measure_states:
    print(f"|{''.join(str(j) for j in i)}>: {probs[i]}")

|20000>: 0.06303464988294274
|02000>: 0.0008393044001482001
|00200>: 0.054218915853727866
|00020>: 0.1046946313741777
|00002>: 0.1046946313741777
|11000>: 0.062482452974773964
|10100>: 0.00030686172379281476
|10010>: 0.18112538056611927
|10001>: 0.007733233898332965
|01100>: 0.0691178151646144
|01010>: 0.03150785595998333
|01001>: 0.10177102097970148
|00110>: 0.03736676347389741
|00101>: 0.14049574512196558
|00011>: 0.04061073725164468


In [69]:
probabilities = [probs[i].item() for i in measure_states]

In [70]:
probs[measure_states[0]]

tensor(0.06303465, requires_grad=True)

In [72]:
probabilities

[0.06567060345865058,
 0.048462743697155114,
 0.008852683037193923,
 0.07594399843073138,
 0.07594399843073142,
 0.01939792151216582,
 0.030659002374392244,
 0.07670283775927167,
 0.10469529304458752,
 0.0769570459204424,
 0.1631766726242217,
 0.030103782202696716,
 0.010120489616506662,
 0.11520092475271582,
 0.09811200313853731]

In [74]:
state_probs /= np.sum(probabilities)

In [73]:
state_probs = probabilities

In [75]:
list(state_probs)

[0.06567060345865057,
 0.0484627436971551,
 0.008852683037193922,
 0.07594399843073137,
 0.07594399843073141,
 0.019397921512165815,
 0.030659002374392237,
 0.07670283775927166,
 0.1046952930445875,
 0.07695704592044239,
 0.16317667262422167,
 0.03010378220269671,
 0.01012048961650666,
 0.11520092475271579,
 0.09811200313853728]

In [82]:
np.random.choice(measure_states_str, p=state_probs)


'10010'

In [48]:
measure_states_str = [''.join(map(str, state)) for state in measure_states]

In [11]:
state_probs = [0.06567060345865057,
 0.0484627436971551,
 0.008852683037193922,
 0.07594399843073137,
 0.07594399843073141,
 0.019397921512165815,
 0.030659002374392237,
 0.07670283775927166,
 0.1046952930445875,
 0.07695704592044239,
 0.16317667262422167,
 0.03010378220269671,
 0.01012048961650666,
 0.11520092475271579,
 0.09811200313853728]

In [12]:
measure_states

[(0, 0, 0, 1, 1),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (0, 0, 0, 0, 2),
 (0, 0, 0, 2, 0),
 (0, 0, 2, 0, 0),
 (0, 2, 0, 0, 0),
 (2, 0, 0, 0, 0)]

In [28]:

[1 if x > 1 else x for x in list(measure_states[np.random.choice(range(len(measure_states)), p=state_probs)])]

[0, 0, 0, 1, 0]

In [6]:
from itertools import groupby, permutations
import numpy as np

def get_subm_idx(inp_state_array):
    indices = []
    for i in range(len(inp_state_array)):
        indices.extend([i] * inp_state_array[i])
    return indices

def decompose(n):
    if n == 1:
        return [[1]]
    elif n == 2:
        return [[1, 1], [2]]
    elif n == 3:
        return [[3], [2, 1], [1, 1, 1]]
    elif n == 4:
        return [[4], [3, 1], [2, 2], [2, 1, 1], [1, 1, 1, 1]]
    elif n == 5:
        return [[5], [4, 1], [3, 1, 1], [3, 2], [2, 1, 1, 1], [2, 2, 1], [1, 1, 1, 1, 1]]
    else:
        raise NotImplemented("Works only for n < 6 :)!")

def generate_output_states(total_photons, dim):
    output_states = []
    for i in decompose(total_photons):
        diff = [0] * (dim - len(i))
        i.extend(diff)
        l = list(permutations(i))
        l.sort()
        cleaned = list(l for l,_ in groupby(l))
        for c in cleaned:
            s = [str(i) for i in c]
            output_states.append("".join(s))
    return output_states

In [7]:
generate_output_states(2, 5)

['00011',
 '00101',
 '00110',
 '01001',
 '01010',
 '01100',
 '10001',
 '10010',
 '10100',
 '11000',
 '00002',
 '00020',
 '00200',
 '02000',
 '20000']

In [4]:
[tuple((i // 2) for i in np.unravel_index(idx, [3] * 5)) for idx in range(5)]

[(0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0)]

In [5]:
from itertools import groupby, permutations
import numpy as np

def get_subm_idx(inp_state_array):
    indices = []
    for i in range(len(inp_state_array)):
        indices.extend([i] * inp_state_array[i])
    return indices

def decompose(n):
    if n == 1:
        return [[1]]
    elif n == 2:
        return [[1, 1], [2]]
    elif n == 3:
        return [[3], [2, 1], [1, 1, 1]]
    elif n == 4:
        return [[4], [3, 1], [2, 2], [2, 1, 1], [1, 1, 1, 1]]
    elif n == 5:
        return [[5], [4, 1], [3, 1, 1], [3, 2], [2, 1, 1, 1], [2, 2, 1], [1, 1, 1, 1, 1]]
    else:
        raise NotImplementedError("Works only for n < 6 :)!")

def generate_output_states(total_photons, dim):
    output_states = []
    for i in decompose(total_photons):
        diff = [0] * (dim - len(i))  # Extend the list to the correct dimension
        i.extend(diff)
        l = list(permutations(i))
        l.sort()
        cleaned = list(l for l, _ in groupby(l))  # Remove duplicate permutations
        for c in cleaned:
            output_states.append(tuple(c))  # Append as tuple instead of string
    return output_states

# Example usage:
total_photons = 2
dim = 5
measure_states = generate_output_states(total_photons, dim)
print(measure_states)


[(0, 0, 0, 1, 1), (0, 0, 1, 0, 1), (0, 0, 1, 1, 0), (0, 1, 0, 0, 1), (0, 1, 0, 1, 0), (0, 1, 1, 0, 0), (1, 0, 0, 0, 1), (1, 0, 0, 1, 0), (1, 0, 1, 0, 0), (1, 1, 0, 0, 0), (0, 0, 0, 0, 2), (0, 0, 0, 2, 0), (0, 0, 2, 0, 0), (0, 2, 0, 0, 0), (2, 0, 0, 0, 0)]


In [8]:
generate_output_states(total_photons, dim)

['00011',
 '00101',
 '00110',
 '01001',
 '01010',
 '01100',
 '10001',
 '10010',
 '10100',
 '11000',
 '00002',
 '00020',
 '00200',
 '02000',
 '20000']

In [9]:
[tuple(map(int, list(item))) for item in generate_output_states(total_photons, dim)]


[(0, 0, 0, 1, 1),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 1, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 1, 0, 0),
 (1, 1, 0, 0, 0),
 (0, 0, 0, 0, 2),
 (0, 0, 0, 2, 0),
 (0, 0, 2, 0, 0),
 (0, 2, 0, 0, 0),
 (2, 0, 0, 0, 0)]

In [31]:
from itertools import groupby, permutations
import numpy as np

def get_subm_idx(inp_state_array):
    indices = []
    for i in range(len(inp_state_array)):
        indices.extend([i] * inp_state_array[i])
    return indices

def decompose(n):
    if n == 1:
        return [[1]]
    elif n == 2:
        return [[1, 1], [2]]
    elif n == 3:
        return [[3], [2, 1], [1, 1, 1]]
    elif n == 4:
        return [[4], [3, 1], [2, 2], [2, 1, 1], [1, 1, 1, 1]]
    elif n == 5:
        return [[5], [4, 1], [3, 1, 1], [3, 2], [2, 1, 1, 1], [2, 2, 1], [1, 1, 1, 1, 1]]
    else:
        raise NotImplemented("Works only for n < 6 :)!")

def generate_output_states(total_photons, dim):
    output_states = []
    for i in decompose(total_photons):
        diff = [0] * (dim - len(i))
        i.extend(diff)
        l = list(permutations(i))
        l.sort()
        cleaned = list(l for l,_ in groupby(l))
        for c in cleaned:
            s = [str(i) for i in c]
            output_states.append("".join(s))
    return output_states

In [32]:
import numpy as np


def generate_haar_unitary(dim):
    """Generate a (dim, dim) Haar-random matrix using the QR decomposition.
    Source: https://pennylane.ai/qml/demos/tutorial_haar_measure.html

    Parameters:
        dim (int): the dimension of the matrix.
    """
    # Step 1
    A, B = np.random.normal(size=(dim, dim)), np.random.normal(size=(dim, dim))
    Z = A + 1j * B

    # Step 2
    Q, R = np.linalg.qr(Z)

    # Step 3
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(dim)])

    # Step 4
    return np.dot(Q, Lambda)


In [37]:
#from \Users\Client\Desktop\cqt_rng2.0\cqt_rng2.0\utils\generate_haar_unitary import generate_haar_unitary
import pennylane as qml
import numpy as np
#from \Users\Client\Desktop\cqt_rng2.0\cqt_rng2.0\utils\utils import generate_output_states



class ShiSFSampler():
    """Simulates the Shi. and al. experiment on Strawberry Fields.

    Parameters:
        nb_modes (int, optional): Number of modes of the interferometer (should be an even number). Default to `6`.
        unitary_top (np.ndarray, optional): The unitary to apply on the top-half of the modes. Default to randomly generated haar matrix.
        unitary_bottom (np.ndarray, optional): The unitary to apply on the bottom-half of the modes. Default to randomly generated haar matrix.
        cut_off_dim = the maximum dimension of the fock space. Default to 4
    """

    def __init__(self, **kwargs):
        self.name = "ShiSFSampler"
        self.nb_modes = kwargs.get("nb_modes")
        self.unitary_top = kwargs.get("unitary_top")
        self.unitary_bottom = kwargs.get("unitary_bottom")
        self.cutoff_dim = kwargs.get("cut_off_dim")
        


        if self.cutoff_dim is None:
            self.cutoff_dim = 3

        if self.nb_modes is None:
            self.nb_modes = 4

        if self.nb_modes % 2 or self.nb_modes < 4:
            raise ValueError(
                "Wrong number of modes (nb_modes) expected to be even and higher than 3!"
            )

        if self.unitary_top is None:
            self.unitary_top = generate_haar_unitary(self.nb_modes // 2)

        if self.unitary_bottom is None:
            self.unitary_bottom = generate_haar_unitary(self.nb_modes // 2)

        if (
            np.shape(self.unitary_top)[0] != self.nb_modes // 2
            or np.shape(self.unitary_bottom)[0] != self.nb_modes // 2
            or np.shape(self.unitary_top)[1] != self.nb_modes // 2
            or np.shape(self.unitary_bottom)[1] != self.nb_modes // 2
        ):
            raise ValueError(
                f"Wrong unitary dimensions expect to be ({int(self.nb_modes / 2)},{int(self.nb_modes / 2)}) for top and bottom!"
            )

        self.dep_seq_len = self.nb_modes
        self.seq_len = self.nb_modes
        self.dev = qml.device("strawberryfields.fock", wires=self.nb_modes, cutoff_dim=self.cutoff_dim)

    def _successful_entanglement(self, sample):
        nb_modes = np.size(sample)
        return (
            np.sum(sample[: nb_modes // 2]) == 1
            and np.sum(sample[nb_modes // 2 :]) == 1
        )

    def _plsf_simulator(self):

        @qml.qnode(self.dev)
        def circuit():
            mid_l = self.nb_modes // 2 - 1
            mid_h = self.nb_modes // 2

            # Two-photon entanglement source
            qml.FockState(1, wires=mid_l)
            qml.FockState(1, wires=mid_h)

            # Beam splitters for entanglement
            qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 4, wires=[mid_l - 1, mid_l])
            qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 4, wires=[mid_h, mid_h + 1])

            # SWAP operation
            qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 2, wires=[mid_l, mid_h])

            # Interferometer
            qml.InterferometerUnitary(self.unitary_top, wires=list(range(mid_l + 1)))
            qml.InterferometerUnitary(self.unitary_bottom, wires=list(range(mid_h, self.nb_modes)))

            # Measurement
            return qml.probs(wires=range(self.nb_modes))
        
        probs = circuit().reshape([self.cutoff_dim] * self.nb_modes)

        measure_states = [tuple(map(int, list(item))) for item in generate_output_states(2, self.cutoff_dim)]

        probabilities = [probs[i].item() for i in measure_states]

        probabilities /= np.sum(probabilities)

        return np.asarray([1 if x > 1 else x for x in list(measure_states[np.random.choice(range(len(measure_states)), p=probabilities)])])
        

    def _run_experiment(self):
        ran_once = False
        while not ran_once or self._successful_entanglement(sample):
            sample = self._plsf_simulator()
            ran_once = True
        return sample

    def sample(self, length):
        shots = length // self.seq_len
        ret = np.array([])
        for _ in range(shots):
            ret = np.append(ret, self._run_experiment())

        return np.copy(ret[:length]).astype(np.int8)

In [40]:
ShiSFSampler().sample(10)

TypeError: 'NotImplementedType' object is not callable

In [85]:
dev = qml.device("strawberryfields.fock", wires=4, cutoff_dim=3)
unitary_top = generate_haar_unitary(4 // 2)
unitary_bottom = generate_haar_unitary(4 // 2)
@qml.qnode(dev)
def circuit():
    mid_l = 4 // 2 - 1
    mid_h = 4 // 2
    
    # Two-photon entanglement source
    qml.FockState(1, wires=mid_l)
    qml.FockState(1, wires=mid_h)
    
    # Beam splitters for entanglement
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 4, wires=[mid_l - 1, mid_l])
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 4, wires=[mid_h, mid_h + 1])
    
    # SWAP operation
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 2, wires=[mid_l, mid_h])
    
    # Interferometer
    qml.InterferometerUnitary(unitary_top, wires=list(range(mid_l + 1)))
    qml.InterferometerUnitary(unitary_bottom, wires=list(range(mid_h, 4)))
    
    # Measurement
    return qml.probs(wires=range(4))

probs = circuit().reshape([3] * 4)
#print(probs.shape)

measure_states = [tuple(map(int, list(item))) for item in generate_output_states(2, 4)]
#print(measure_states)


probabilities = [probs[i].item() for i in measure_states]

probabilities /= np.sum(probabilities)

np.asarray([1 if x > 1 else x for x in list(measure_states[np.random.choice(range(len(measure_states)), p=probabilities)])])

array([1, 0, 0, 1])